# Demand Models

We consider three approaches to model demand (board and alight counts):

1. Random Forest-Random Forest: We train a random forest model to classify zero counts, and we also train a second random forest model for counts.
2. Classification: Instead of modeling demand as a regression problem, we build three different labels to indicate low, average, and high demand. Then, we train tree-based methods to model (classify) these labels.
3. Neural Networks and Zero Inflated models.

In [1]:
import numpy as np
import pandas as pd
import timeit
import pickle

In [2]:
carta = pd.read_csv('Data/Carta_out.csv')

In [3]:
carta.stop_id = carta['stop_id'].astype(str)
carta.stop_sequence = carta['stop_sequence'].astype(int)
carta.route_id = carta['route_id'].astype(str)
carta.direction_id = carta['direction_id'].astype(str)
carta.service_period = carta['service_period'].astype(str)
carta.actual_arrival_time = pd.to_datetime(carta['actual_arrival_time'], format = "%Y-%m-%d %H:%M:%S")
carta.scheduled_arrival_time = pd.to_datetime(carta['scheduled_arrival_time'], format = "%Y-%m-%d %H:%M:%S")
carta.date = pd.to_datetime(carta['date'], format = "%Y-%m-%d")
carta.scheduled_datetime = pd.to_datetime(carta['scheduled_datetime'], format = "%Y-%m-%d %H:%M:%S")
carta.trip_date = pd.to_datetime(carta['trip_date'], format="%Y-%m-%d")
carta.trip_start_time = pd.to_datetime(carta['trip_start_time'], format="%Y-%m-%d %H:%M:%S")

In [4]:
carta['year'] = carta['date'].dt.year
carta['month'] = carta['date'].dt.month

In [5]:
# Route 9 - Stop 1710
# Route 4 - Stop 1351

In [6]:
carta[(carta['route_id'] == '4') & (carta['stop_id'] == '12') & (carta['direction_id'] == '1')].drop_duplicates().shape

(5619, 25)

In [7]:
service_period = carta['service_period']

service_kind = []

for i in service_period:
    if (i == 'Saturday'):
        kind = 'weekend'
    elif (i == 'Sunday'):
        kind = 'weekend'
    else:
        kind = 'weekday'
        
    service_kind.append(kind)

In [8]:
carta['service_kind'] = service_kind

In [9]:
carta.head()

,trip_id,scheduled_arrival_time,actual_arrival_time,stop_id,stop_sequence,stop_lat,stop_lon,route_id,direction_id,board_count,...,actual_arrival_datetime,trip_start_time,day_of_week,trip_date,hour,year,month,Estimated_Temp,Estimated_Precip,service_kind
0,134313,1900-01-01 04:21:00,1900-01-01 04:22:00,354,1,35.056167,-85.268713,4,0,0,...,2019-01-02 04:22:00,1900-01-01 04:21:00,2,2019-01-02,4,2019,1,8.690000,0.0,weekday
1,134313,1900-01-01 04:23:51,1900-01-01 04:24:00,505,2,35.056017,-85.281080,4,0,0,...,2019-01-02 04:24:00,1900-01-01 04:21:00,2,2019-01-02,4,2019,1,8.661500,0.0,weekday
2,134360,1900-01-01 04:26:00,1900-01-01 04:25:00,354,1,35.056167,-85.268713,4,0,0,...,2019-01-02 04:25:00,1900-01-01 04:26:00,2,2019-01-02,4,2019,1,8.640000,0.0,weekday
3,134313,1900-01-01 04:28:22,1900-01-01 04:27:00,284,5,35.052515,-85.302427,4,0,0,...,2019-01-02 04:27:00,1900-01-01 04:21:00,2,2019-01-02,4,2019,1,8.616333,0.0,weekday
4,134313,1900-01-01 04:28:57,1900-01-01 04:27:00,285,6,35.053513,-85.305272,4,0,0,...,2019-01-02 04:27:00,1900-01-01 04:21:00,2,2019-01-02,4,2019,1,8.610500,0.0,weekday


In [10]:
carta.shape

(9705963, 26)

In [11]:
carta.dtypes

trip_id                             int64
scheduled_arrival_time     datetime64[ns]
actual_arrival_time        datetime64[ns]
stop_id                            object
stop_sequence                       int64
stop_lat                          float64
stop_lon                          float64
route_id                           object
direction_id                       object
board_count                         int64
alight_count                        int64
occupancy                           int64
direction_desc                     object
service_period                     object
date                       datetime64[ns]
scheduled_datetime         datetime64[ns]
actual_arrival_datetime            object
trip_start_time            datetime64[ns]
day_of_week                         int64
trip_date                  datetime64[ns]
hour                                int64
year                                int64
month                               int64
Estimated_Temp                    

In [12]:
carta['route_id'].unique()

array(['4', '7', '13', '9', '1', '8', '10G', '16', '15', '10C', '2', '28',
       '19', '10A', '21', '3'], dtype=object)

In [13]:
carta[carta['route_id'] == '4']['stop_id'].unique()

array(['354', '505', '284', '285', '713', '286', '287', '1351', '12',
       '1555', '1579', '1354', '1353', '17', '742', '805', '806', '807',
       '1475', '1474', '811', '812', '813', '814', '815', '816', '817',
       '818', '819', '820', '821', '822', '823', '824', '825', '826',
       '827', '828', '829', '321', '322', '830', '831', '832', '833',
       '834', '835', '836', '837', '838', '839', '840', '841', '842',
       '843', '844', '845', '846', '848', '849', '850', '851', '852',
       '854', '892', '855', '1875', '856', '894', '784', '857', '858',
       '283', '859', '897', '860', '861', '898', '1873', '1848', '1849',
       '1850', '1819', '867', '1877', '868', '869', '870', '871', '923',
       '872', '924', '925', '873', '874', '875', '876', '877', '926',
       '927', '879', '928', '880', '881', '929', '930', '931', '882',
       '932', '883', '933', '884', '934', '885', '886', '887', '888',
       '935', '889', '890', '936', '891', '1578', '1485', '939', '941',
      

In [14]:
min(carta.scheduled_datetime)

Timestamp('2019-01-02 04:21:00')

## Subordinate Functions

In [15]:
def normalizer(x):
    
    x_min = min(x)
    x_max = max(x)
    x_norm = (x - x_min)/(x_max - x_min)
    
    return(x_norm)

In [16]:
def standardizer(x):
    x_std = x.std()
    x_mean = x.mean()
    x_standard = (x - x_mean)/x_std
    
    return(x_standard)

## Surrounding Board or Aligh Counts

In [17]:
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
import geopandas as gpd
from geopy import Point
from geopy import distance

In [18]:
Bus_Stops = pd.read_csv('Bus_Stops.csv')
Bus_Stops.stop_id = Bus_Stops['stop_id'].astype(str)
Bus_Stops.head()

,stop_id,stop_lon,stop_lat
0,971,-85.246812,35.024347
1,146,-85.304620,34.989585
2,1545,-85.250863,35.026032
3,972,-85.248528,35.025717
4,81,-85.304890,34.990302


### Radial Influence

In [19]:
def radial_influence(st, radius, DT):
    n_row = DT.shape[0]
    
    stopd_index = DT[DT['stop_id'] == st].index[0]
    x0 = DT.stop_lon.iloc[stopd_index]
    y0 = DT.stop_lat.iloc[stopd_index]
    
    radial_dist = []
    condition = []
    
    for i in range(n_row):
        
        center_point = (x0, y0)
        test_point = (DT.stop_lon.iloc[i], DT.stop_lat.iloc[i])
        dist = distance.distance(center_point, test_point).miles
        
        radial_dist.append(dist)
        
        if (dist <= radius):
            
            cond = 'inside'
            
        else:
            
            cond = 'outside'
            
        condition.append(cond)
        
    influence = {'stop_id': DT.stop_id, 'Radial_Distance': radial_dist, 'Influence': condition}
    Influence = pd.DataFrame(data = influence, columns = ['stop_id', 'Radial_Distance', 'Influence'])
    
    return(Influence)

## Data Extraction

In [77]:
def data_extraction(route, direction, bus_stop, DTFRM):
    import numpy as np
    import pandas as pd
    import datetime as dt
             
    dtfrm = DTFRM[(DTFRM['route_id'] == route) & (DTFRM['direction_id'] == direction) & (DTFRM['stop_id'] == bus_stop)]
    
    n_rows = dtfrm.shape[0]
    
    if (n_rows == 0):
        print('There are not rows in the data set with the required characteristics. Please change them.')
        
    else:
          
        vinit = dtfrm[['month', 'service_kind', 'hour', 'Estimated_Temp', 'Estimated_Precip']].groupby(['month', 'service_kind', 'hour']).mean()
        Vinit = vinit.reset_index(level = ['month', 'service_kind', 'hour'])
        Vinit.columns = ['month', 'service_kind', 'hour', 'mean_temp', 'mean_precip']
        Vinit = Vinit.drop_duplicates()
    
        V1 = dtfrm[[ 'month', 'service_kind', 'hour', 'board_count', 'alight_count']]
        V1.columns = ['month', 'service_kind', 'hour', 'board_count', 'alight_count']
     
        V1 = pd.merge(V1, vinit, on = ['month', 'service_kind', 'hour'], how = 'left')
    
        Relevant_Bus_Stops = radial_influence(bus_stop, 0.5, Bus_Stops)
        bus_stops_ids = Relevant_Bus_Stops.stop_id.unique().astype(str)
        bus_stops_ids = pd.DataFrame(bus_stops_ids)
        bus_stops_ids.columns = ['stop_id']
        bus_stops_ids = bus_stops_ids[bus_stops_ids['stop_id'] != bus_stop]
        bus_stops_ids = bus_stops_ids.drop_duplicates()
               
        V3 = DTFRM[DTFRM['stop_id'].isin(bus_stops_ids.stop_id.astype(str))][['month', 'service_kind', 'hour', 'board_count', 'alight_count']]
        V3 = V3.drop_duplicates()
        V3a = V3.groupby(['month', 'service_kind', 'hour']).mean()
        V3b = V3a.reset_index(level = ['month', 'service_kind', 'hour'])
        V3b.columns = ['month', 'service_kind', 'hour', 'surrounding_board_count', 'surrounding_alight_count']
        V3b = V3b.drop_duplicates()
    
        data = pd.merge(V1, V3b, how = 'left', on = [ 'month', 'service_kind', 'hour'])
                
        data.month = data['month'].astype('category')
        data.service_kind = data['service_kind'].astype('category')
        data.hour = data['hour'].astype('category')
       
        data.columns = ['month', 'service_kind', 'hour', 'board_count', 'alight_count', 'mean_temp', 'mean_precip', 'surrounding_board_count', 'surrounding_alight_count']
        data = data.drop_duplicates()
        
        data.surrounding_board_count = normalizer(data['surrounding_board_count'])
        #V.surrounding_alight_count = normalizer(V['surrounding_alight_count'])
        data.mean_temp = normalizer(data['mean_temp'])
        data.mean_precip = normalizer(data['mean_precip'])
        
        data = data[['month', 'service_kind', 'hour', 'board_count', 'mean_temp', 'mean_precip', 'surrounding_board_count']]
        
        data = data.reset_index(drop = True)
        
        #--------------------------------------------------------------------------------------------------------------------
        # Export data:
        
        def paste0(ss,sep=None,na_rep=None,):
            '''Analogy to R paste0'''
            ss = [pd.Series(s) for s in ss]
            ss = [s.astype(str) for s in ss]
            s = ss[0]
            res = s.str.cat(ss[1:],sep=sep,na_rep=na_rep)
            return res
        
        file_path = paste0([paste0([paste0(['Data_for_RF_Models', 'Board_Counts'], sep ='/'), 
                                    paste0(['route', route], sep = '_')], sep = '/'), 
                            paste0([paste0(['direction', direction]),
                                    paste0(['bus_stop', bus_stop], sep = '_')], sep = '/')], sep = '/')
        
        file_name = paste0(['data', '.csv'])
        
        complete_path = paste0([file_path, file_name], sep = '/')[0]
        
        data.to_csv(complete_path)
        
        return(data)

In [78]:
data_extraction('4', '1', '12', carta)

,month,service_kind,hour,board_count,mean_temp,mean_precip,surrounding_board_count
0,1,weekday,9,0,0.065536,0.009611,0.616041
1,1,weekday,10,1,0.127482,0.021835,0.662874
2,1,weekday,13,0,0.211614,0.011898,0.785362
3,1,weekday,14,0,0.226468,0.020642,0.813413
4,1,weekday,15,0,0.198916,0.017472,0.719390
...,...,...,...,...,...,...,...
1392,5,weekday,23,2,0.568541,0.064658,0.559300
1393,5,weekday,15,1,0.676908,0.007262,0.813769
1394,5,weekday,21,0,0.647474,0.000000,0.852610
1395,5,weekend,12,0,0.714363,0.000000,0.672782


In [18]:
board_count_route4_dir1_stop12 = data_extraction('4', '1', '12', carta)

In [19]:
board_count_route4_dir1_stop12.shape

(1397, 7)

In [79]:
board_count_route4_dir1_stop1351 = data_extraction('4', '1', '1351', carta)